In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from PIL import ImageFilter
import multiprocessing
import random; random.seed(2016);




import cv2
import re
import os, glob

sample_sub = pd.read_csv('../input/sample_submission.csv')
train_files = pd.DataFrame([[f,f.split("/")[3].split(".")[0].split("_")[0],f.split("/")[3].split(".")[0].split("_")[1]] for f in glob.glob("../input/train_sm/*.jpeg")])
train_files.columns = ['path', 'group', 'pic_no']
test_files = pd.DataFrame([[f,f.split("/")[3].split(".")[0].split("_")[0],f.split("/")[3].split(".")[0].split("_")[1]] for f in glob.glob("../input/test_sm/*.jpeg")])
test_files.columns = ['path', 'group', 'pic_no']
print(len(train_files),len(test_files),len(sample_sub))




In [ ]:
test_files["group"].unique()

In [ ]:
b = []
for name in test_files["group"].unique():
    test_images = test_files[test_files["group"]==name]
    test_images = test_images.sort_values(by=["pic_no"], ascending=[1])
    plt.rcParams['figure.figsize'] = (12.0, 12.0)
    plt.subplots_adjust(wspace=0, hspace=0)
    i_ = 0
    a = []
    for l in test_images.path:
        im = cv2.imread(l)
        #plt.subplot(5, 2, i_+1).set_title(l)
        #plt.hist(im.ravel(),256,[0,256]); plt.axis('off')
        a.append(im.mean())
        #plt.subplot(5, 2, i_+2).set_title(l)
        #plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
        i_ += 2
    b.append(a)
print(b)

In [ ]:
import pandas as pd
df= pd.DataFrame(b)
df

In [ ]:
df.to_csv("immediantest.csv")

In [ ]:
im = Image.open(train_images.path.iloc[0])
im2 = Image.open(train_images.path.iloc[1])
im2 = im2.rotate(-15)
im2 = im2.resize((3000, 2000), Image.ANTIALIAS)
r,g,b = im2.split()
mask = Image.merge("L", (b,))
im.paste(im2, (0,0,3000,2000), mask)
plt.imshow(im); plt.axis('off')

In [ ]:
train_images = train_files[train_files["group"]=='set12']
train_images = train_images.sort_values(by=["pic_no"], ascending=[1])
plt.rcParams['figure.figsize'] = (
12.0, 12.0)
plt.subplots_adjust(wspace=0, hspace=0)
i_ = 0
a = []
for l in train_images.path:
    im = cv2.imread(l)
    plt.subplot(5, 2, i_+1).set_title(l)
    plt.hist(im.ravel(),256,[0,256]); plt.axis('off')
    a.append([im.mean(),im.max(),im.min()])
    plt.subplot(5, 2, i_+2).set_title(l)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 2
print(a)

In [ ]:
# from https://www.kaggle.com/nigelcarpenter/draper-satellite-image-chronology/akaze-stitching/run/227957
import numpy as np
import cv2

print(cv2.__version__)

def warpImage(img, H):
	rows, cols = img.shape[:2]

	list_of_points_1 = np.float32([[0,0], [0,rows], [cols, rows], [cols,0]]).reshape(-1,1,2)
	list_of_points = cv2.perspectiveTransform(list_of_points_1, H)

	[x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
	[x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)

	print("x_min: "+str(x_min)+" y_min:"+str(y_min))
	translation_dist = [-x_min, -y_min]
	H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0,0,1]])
	output_img = cv2.warpPerspective(img, H_translation.dot(H), (x_max - x_min, y_max - y_min))
	return output_img

def warpImages(img1, img2, H):
	rows1, cols1 = img1.shape[:2]
	rows2, cols2 = img2.shape[:2]

	list_of_points_1 = np.float32([[0,0], [0,rows1], [cols1, rows1], [cols1,0]]).reshape(-1,1,2)
	temp_points = np.float32([[0,0], [0,rows2], [cols2, rows2], [cols2,0]]).reshape(-1,1,2)

	list_of_points_2 = cv2.perspectiveTransform(temp_points, H)
	list_of_points = np.concatenate((list_of_points_1, list_of_points_2), axis=0)

	[x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
	[x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)

	translation_dist = [-x_min, -y_min]
	H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0,0,1]])

	output_img = cv2.warpPerspective(img2, H_translation.dot(H), (x_max - x_min, y_max - y_min))
	output_img[translation_dist[1]:rows1+translation_dist[1],translation_dist[0]:cols1+translation_dist[0]] = img1
	return output_img



def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
	# initialize the output visualization image
	(hA, wA) = imageA.shape[:2]
	(hB, wB) = imageB.shape[:2]
	vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
	vis[0:hA, 0:wA] = imageA
	vis[0:hB, wA:] = imageB

	# loop over the matches
	for ((trainIdx, queryIdx), s) in zip(matches, status):
		# only process the match if the keypoint was successfully
		# matched
		if s == 1:
			# draw the match
			ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
			ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
			cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

	# return the visualization
	return vis

def detectAndDescribe(image):
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	descriptor = cv2.AKAZE_create()
	(kps, features) = descriptor.detectAndCompute(gray, None)

	# convert the keypoints from KeyPoint objects to NumPy
	# arrays
	kps = np.float32([kp.pt for kp in kps])

	# return a tuple of keypoints and features
	return (kps, features)

def matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
	# compute the raw matches and initialize the list of actual
	# matches
	matcher = cv2.DescriptorMatcher_create("BruteForce")
	rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
	matches = []

	# loop over the raw matches
	for m in rawMatches:
		# ensure the distance is within a certain ratio of each
		# other (i.e. Lowe's ratio test)
		if len(m) == 2 and m[0].distance < m[1].distance * ratio:
			matches.append((m[0].trainIdx, m[0].queryIdx))
		# computing a homography requires at least 4 matches
		if len(matches) > 30:
			# construct the two sets of points
			ptsA = np.float32([kpsA[i] for (_, i) in matches])
			ptsB = np.float32([kpsB[i] for (i, _) in matches])
 
			# compute the homography between the two sets of points
			(H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
				reprojThresh)
 
			# return the matches along with the homograpy matrix
			# and status of each matched point
			return (matches, H, status)
 
	# otherwise, no homograpy could be computed
	return None



In [ ]:
import time; start_time = time.time()
import warnings; warnings.filterwarnings('ignore');
import multiprocessing
from sklearn import ensemble
from sklearn import grid_search
from sklearn.metrics import label_ranking_average_precision_score as lraps

def image_features(path, tt, group, pic_no):
    im = cv2.imread(path)
    s=[path, tt, group, pic_no, im.mean(), 0.0, 0.0, 0.0]
    f = open("data.csv","a")
    f.write((',').join(map(str, s)) + '\n')
    f.close()
    return

f = open("data.csv","w");
col = ['path','tt', 'group', 'pic_no', 'individual_im_mean', 'group_min_im_mean', 'group_max_im_mean', 'group_mean']
f.write((',').join(map(str,col)) + '\n')
f.close()

if __name__ == '__main__':
    cpu = multiprocessing.cpu_count(); print (cpu);
    
    j = []
    for s_ in range(0,len(train_files),cpu):     #train
        for i in range(cpu):
            i_=s_+i
            if (i_)<len(train_files):
                if i_ % 100 == 0:
                    print("train ", i_)
                filename = train_files.path[i_]
                p = multiprocessing.Process(target=image_features, args=(filename,'train', train_files["group"][i_], train_files["pic_no"][i_],))
                j.append(p)
                p.start()
    j = []
    for s_ in range(0,len(test_files),cpu):     #test
        for i in range(cpu):
            i_=s_+i
            if (i_)<len(test_files):
                if i_ % 100 == 0:
                    print("test ", i_)
                filename = test_files.path[i_]
                p = multiprocessing.Process(target=image_features, args=(filename,'test', test_files["group"][i_], test_files["pic_no"][i_],))
                j.append(p)
                p.start()
    
    while len(j) > 0: #end all jobs
        j = [x for x in j if x.is_alive()]
        time.sleep(1)
        
    df_all = pd.read_csv('data.csv', index_col=None)
    df_all = df_all.reset_index(drop=True)
    df_all['group_min_im_mean'] = df_all["group"].map(lambda x: df_all[df_all['group']==x]['individual_im_mean'].min())
    df_all['group_max_im_mean'] = df_all["group"].map(lambda x: df_all[df_all['group']==x]['individual_im_mean'].max())
    df_all['group_mean'] = df_all["group"].map(lambda x: df_all[df_all['group']==x]['individual_im_mean'].mean())
    df_all['a'] = df_all['individual_im_mean'] - df_all['group_min_im_mean']
    df_all['b'] = df_all['group_max_im_mean'] - df_all['individual_im_mean']
    df_all['c'] = df_all['group_mean'] - df_all['individual_im_mean']
    df_all['setId'] = df_all["group"].map(lambda x: x.replace('set',''))
    df_all.to_csv('data.csv')
    print("Features Ready: ", round(((time.time() - start_time)/60),2))
    
    X_train = df_all[df_all['tt'] == 'train']
    X_train = X_train.sort_values(by=['setId','pic_no'], ascending=[1, 1])
    X_train = X_train.reset_index(drop=True)
    y_train = X_train["pic_no"].values
    X_train = X_train.drop(['path','tt','group','pic_no','setId','individual_im_mean','group_min_im_mean','group_max_im_mean','group_mean'], axis=1)
    X_test = df_all[df_all['tt'] == 'test']
    X_test = X_test.sort_values(by=['setId','pic_no'], ascending=[1, 1])
    #X_test.fillna(0, inplace=True)
    X_test = X_test.reset_index(drop=True)
    id_test = X_test[["setId","pic_no"]] #.values
    X_test = X_test.drop(['path','tt','group','pic_no','setId','individual_im_mean','group_min_im_mean','group_max_im_mean','group_mean'], axis=1)
    rfr = ensemble.RandomForestClassifier(n_estimators = 50, n_jobs = -1, random_state = 2016, verbose = 0)
    param_grid = {'max_depth': [6], 'max_features': [3]}
    model = grid_search.GridSearchCV(estimator = rfr, param_grid = param_grid, n_jobs = -1, cv = 2, verbose = 0)
    model.fit(X_train, y_train)
    print("Best parameters found by grid search:")
    print(model.best_params_)
    print("Best CV score:", model.best_score_)
    y_pred = model.predict_proba(X_test)
    #y_pred = model.predict(X_test)
    df = pd.concat((pd.DataFrame(id_test), pd.DataFrame(y_pred)), axis=1)
    df.columns = ['setId','pic_no','day1','day2','day3','day4','day5']
    #df.to_csv('submission2.csv',index=False)
    f = open('submission.csv', 'w')
    f.write('setId,day\n')
    setID = df.setId.unique()
    for i in setID:
        a = []
        df1 = df[df['setId'] == str(i)].reset_index(drop=True)
        for j in range(1,6):
            df1 = df1.sort_values(by=['day'+str(j)], ascending=[0]).reset_index(drop=True)
            #print(df1)
            a.append(df1.pic_no[0])
            df1 = df1[1:]
        f.write(str(i)+","+" ".join(map(str,a))+"\n")
        #break
    f.close()
    print("Ready to submit: ", round(((time.time() - start_time)/60),2))